In [ ]:
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.master("local[*]") \
    .enableHiveSupport().appName("ex01_1") \
    .getOrCreate()

spark.version   # 2.4.5

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.master("local[*]") \
    .appName("mg01") \
    .config('spark.jars.packages','org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5' ) \
    .getOrCreate()

df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers","127.0.0.1:9092") \
    .option("subscribe", "testTopic1") \
    .option("startingOffsets", "latest") \
    .load()

#df의 값을 변경해서 df1에 보관
df1 = df.selectExpr("CAST (key AS STRING)", "CAST(value AS STRING)")

#df1의 실시간 값을 console에 출력함
df1.writeStream.outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime="3 seconds") \
    .start() \
    .awaitTermination()


In [ ]:
pip install pymongo

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import sum, count, avg, expr, split
import pymongo # pip install pymongo

# 파이썬 전용
# conn = pymongo.MongoClient('192.168.99.100',32888)
# db = conn.get_database("db1")
# tb = db.get_collection("t07") #collection 생성

spark = SparkSession.builder.master("local[*]") \
    .appName("mg01") \
    .config('spark.jars.packages','org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5' ) \
    .config("spark.mongodb.output.uri", "mongodb://192.168.99.100:32888/db1.t07") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.4.1') \
    .getOrCreate()

df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers","127.0.0.1:9092") \
    .option("subscribe", "testTopic1") \
    .option("startingOffsets", "latest") \
    .load()

#df의 값을 변경해서 df1에 보관
df1 = df.selectExpr("CAST (key AS STRING)", "CAST(value AS STRING)")

def process_row(df_tmp, epoch_id):
    df2 = df_tmp.select("value")
    split_col = split(df2['value'], ',')
    
    df3 = df2.withColumn("num0", split_col[0])
    df3 = df3.withColumn("num1", split_col[1])
    df3 = df3.withColumn("num2", split_col[2])
    df3 = df3.withColumn("num3", split_col[3])
    df3.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").save(); # 실시간으로 데이터 저장
    df3.show()
    

df1.writeStream.foreachBatch(process_row).start().awaitTermination()

